In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('drive/My Drive/Colab Notebooks/UMich/SI670/Final_Proj')

Mounted at /content/drive


In [ ]:
!ls

 calorie_regression_baseline.ipynb
 calorie_regression_basic_densenet121.ipynb
 calorie_regression_multiin_101_multiout.ipynb
 calorie_regression_multiin_emb_multiout_densenet121.ipynb
 calorie_regression_multiin_multiout_densenet121.ipynb
 calorie_regression_multiout_densenet121.ipynb
 clean_recipe_data_nutrition.csv
'EfficientNetV2 calorie_regression_multiin_101_multiout.ipynb'
 file.txt
 food-101
 food101_id2label.json
 food101_label2id.json
 food101_nutritionix.csv
 food-101.tar.gz
 get_recipe_images.ipynb
 huggingface_food101_swin.ipynb
 initial_multiin_101_multiout_densenet.h5
 initial_multiin_101_multiout_efficientnetb0.h5
 initial_multiin_101_multiout_mobilenet.h5
 initial_multiin_101_multiout_NASNetMobile.h5
 initial_multiin_multiout_densenet_2.h5
 initial_multiin_multiout_densenet.h5
'mobileNetV2 calorie_regression_multiin_101_multiout.ipynb'
 My_suffering.ipynb
'NASNetMobile calorie_regression_multiin_101_multiout.ipynb'
 no_cat_calorie_regression_multiin_101_multiout.ipynb


In [ ]:
import pandas as pd
from tqdm import tqdm
import requests
import cv2
import numpy as np
import tensorflow as tf
from keras.applications.densenet import DenseNet121, preprocess_input
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image
import zipfile
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
SEED = 670

# Load Data and Preprocess

Basic preprocessing, fix columns, remove unneeded columns, replace NAs with UNK

In [ ]:
data = pd.read_csv('clean_recipe_data_nutrition.csv')
data = data[
    ['id', 'servings', 'calories_per_serving', 'protein_per_serving', 
     'fat_per_serving', 'carb_per_serving', 'meal_type', 'region', 'subregion']
]
data['meal_type'] = data['meal_type'].fillna(value='UNK_meal_type')
data['subregion'] = data['subregion'].fillna(value='UNK_subregion') 
data.head()

,id,servings,calories_per_serving,protein_per_serving,fat_per_serving,carb_per_serving,meal_type,region,subregion
0,1,2,388,17,22,29,main,latin_american,mexican
1,3,10,795,16,39,96,dessert,latin_american,mexican
2,4,12,384,17,25,25,main,latin_american,mexican
3,8,8,381,34,22,9,UNK_meal_type,latin_american,carribean
4,9,12,427,18,12,63,main,latin_american,mexican


# Load images, split data into train and test

In [ ]:
images = []
imgzip = zipfile.ZipFile("Recipe_Images_224_x_224.zip")
for f in tqdm(imgzip.infolist()):
    ifile = imgzip.open(f)
    img = np.asarray(Image.open(ifile))
    images.append(img)


100%|██████████| 8573/8573 [00:22<00:00, 384.43it/s]


In [ ]:
train_df, test_df = train_test_split(data, test_size = 0.2, random_state = SEED)
train_df.head()

,id,servings,calories_per_serving,protein_per_serving,fat_per_serving,carb_per_serving,meal_type,region,subregion
4971,8160,4,127,4,9,9,salad,european,italian
2573,4249,6,300,6,12,44,side,asian,indian
786,1354,8,395,11,17,48,UNK_meal_type,european,"scandi,vian"
3684,6038,25,318,26,6,41,UNK_meal_type,european,"scandi,vian"
435,784,35,141,7,8,11,UNK_meal_type,latin_american,mexican


In [ ]:
train_idx = train_df['id'].values
test_idx = test_df['id'].values
train_images = np.array([images[idx] for idx in train_idx])
test_images = np.array([images[idx] for idx in test_idx])

In [ ]:
train_y = train_df['calories_per_serving'].values
test_y = test_df['calories_per_serving'].values

# Create model

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
IMG_SHAPE = (224, 224, 3)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
base_model.trainable = False

29084464/29084464 [==============================] - 2s 0us/step


In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))
augmented_inputs = data_augmentation(inputs)
processed_out = preprocess_input(augmented_inputs)
base_out = base_model(processed_out, training = False)
flatten_out = GlobalAveragePooling2D()(base_out)
pool_out = Dropout(0.2)(flatten_out)
outputs = Dense(1, activation='linear')(pool_out)
model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = 'huber_loss',
    metrics = ['mean_absolute_error']
)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_2 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.nn.bias_add_1 (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 tf.math.truediv_3 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                           

In [ ]:
es = EarlyStopping(monitor='val_mean_absolute_error', patience=3, restore_best_weights=True)
initial_epochs = 100
history = model.fit(
    x = train_images,
    y = train_y,
    batch_size = 32,
    epochs = initial_epochs,
    validation_split = 0.2,
    callbacks = [es]
)

Epoch 1/100
105/105 [==============================] - 24s 172ms/step - loss: 324.2860 - mean_absolute_error: 324.7859 - val_loss: 302.6392 - val_mean_absolute_error: 303.1386
Epoch 2/100
105/105 [==============================] - 14s 138ms/step - loss: 284.5511 - mean_absolute_error: 285.0507 - val_loss: 266.3615 - val_mean_absolute_error: 266.8615
Epoch 3/100
105/105 [==============================] - 15s 141ms/step - loss: 250.0312 - mean_absolute_error: 250.5304 - val_loss: 237.3386 - val_mean_absolute_error: 237.8386
Epoch 4/100
105/105 [==============================] - 14s 136ms/step - loss: 223.3822 - mean_absolute_error: 223.8815 - val_loss: 216.1343 - val_mean_absolute_error: 216.6338
Epoch 5/100
105/105 [==============================] - 14s 134ms/step - loss: 203.6401 - mean_absolute_error: 204.1394 - val_loss: 199.8586 - val_mean_absolute_error: 200.3568
Epoch 6/100
105/105 [==============================] - 14s 133ms/step - loss: 190.0314 - mean_absolute_error: 190.5309 -

In [ ]:
test_preds = model.predict(test_images)

33/33 [==============================] - 6s 130ms/step


In [ ]:
test_preds

array([[282.1942 ],
       [310.40286],
       [287.1711 ],
       ...,
       [316.7046 ],
       [306.33005],
       [323.0535 ]], dtype=float32)

In [ ]:
mean_absolute_error(test_y, test_preds)

169.51591674395448